#Ellipsoid foundary
This is code to produce ellipsoids to test polymer folding on a simple geometries. The code starts from a sphere which is scaled to have specific a, b, c parametres:

$\frac {x^2} {a^2} + \frac {y^2} {b^2} + \frac {z^2} {c^2} = 1$.

To make the sphere, I started with an icosahedron (42 vertices), and applied the Sqrt(3) subdivision algorithm a few times. The subdivided sphere has 9722 vertices with a pretty good valence. The vertices after Sqrt(3) subdivision are not exactly over the sphere, so I normalised them. I used meshgeometry to produce the sphere, using the following command:

    ./meshgeometry_mac -i ico.ply -subdivide -subdivide -subdivide -subdivide -subdivide -normalise -verts -scale 0.00620463 -volume -o sphere.ply

The following code opens and saves PLY meshes.

result can be saved in PLY or STL formats. The code is generic, and if a mesh other than a sphere is loaded, it could allow to test the effect of x, y, z scaling on folding.

In [1]:
%pylab inline
import numpy
import pylab

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Open mesh in ply format
def open_ply(filepath):
    f=open(filepath,'r');
    i=0;
    ip=0;
    it=0;
    np=0;
    nt=0;
    for str in f:
        i+=1;
        arr=str.split(" ");
        if(arr[0]=="element" and arr[1]=="vertex"):
            np=int(arr[2]);
            p=numpy.zeros((np,3));
        elif(arr[0]=="element" and arr[1]=="face"):
            nt=int(arr[2]);
            t=numpy.zeros((nt,3));
        elif(i>=11 and i<11+np):
            p[ip,0]=float(arr[0]);
            p[ip,1]=float(arr[1]);
            p[ip,2]=float(arr[2]);
            ip+=1;
        elif(i>=11+np and i<11+np+nt):
            t[it,0]=int(arr[1]);
            t[it,1]=int(arr[2]);
            t[it,2]=int(arr[3]);
            it+=1;
    mesh={};
    mesh["np"]=np;
    mesh["nt"]=nt;
    mesh["p"]=p;
    mesh["t"]=t;
    return mesh;

# Save mesh in ply format
def save_ply(mesh,filepath):
    p=mesh["p"];
    t=mesh["t"];
    np=mesh["np"];
    nt=mesh["nt"];
    f=open(filepath,'w');
    f.write("%i 1\n"%np);
    f.write("ply\n");
    f.write("format ascii 1.0\n");
    f.write("comment ipython ferret, R. Toro 2015\n");
    f.write("element vertex %i\n"%np);
    f.write("property float x\n");
    f.write("property float y\n");
    f.write("property float z\n");
    f.write("element face %i\n"%nt);
    f.write("property list uchar int vertex_indices\n");
    f.write("end_header\n");
    for i in range(0,np):
        f.write("%f %f %f\n"%(p[i,0],p[i,1],p[i,2]));
    for i in range(0,nt):
        f.write("3 %i %i %i\n"%(t[i,0],t[i,1],t[i,2]));
    f.close();

# Triangle normal
# Normal of triangle a,b,c
def normal(a,b,c):
    v=numpy.cross(b-a,c-a)
    n=v/numpy.linalg.norm(v)
    return n;

# Save mesh in stl format
def save_stl(mesh,filepath):
    p=mesh["p"];
    t=mesh["t"];
    np=mesh["np"];
    nt=mesh["nt"];
    
    f=open(filepath,'w');
    f.write("solid mySolid\n");
    for i in range(0,nt):
        n=normal(p[t[i,0]],p[t[i,1]],p[t[i,2]]);
        f.write("facet normal %g %g %g\n"%(n[0],n[1],n[2]));
        f.write("outer loop\n");
        f.write("vertex %g %g %g\n"%(p[t[i,0],0],p[t[i,0],1],p[t[i,0],2]));
        f.write("vertex %g %g %g\n"%(p[t[i,1],0],p[t[i,1],1],p[t[i,1],2]));
        f.write("vertex %g %g %g\n"%(p[t[i,2],0],p[t[i,2],1],p[t[i,2],2]));
        f.write("endLoop\n");
        f.write("endFacet\n");
    f.write("endSolid mySolid\n");
    f.close();


In [3]:
# Scale the x, y and z axis of a mesh by a, b and c factors
def scale_mesh(mesh,a,b,c):
    np=mesh["np"];
    nt=mesh["nt"];
    t=mesh["t"];
    p0=mesh["p"];

    newMesh={};
    newMesh["np"]=np;
    newMesh["nt"]=nt;
    newMesh["p"]=numpy.zeros((np,3));
    newMesh["t"]=t; # copy the triangles (because the topology does not change)
    p1=newMesh["p"];
    
    for i in range(0,np):
        p1[i,0]=a*p0[i,0];
        p1[i,1]=b*p0[i,1];
        p1[i,2]=c*p0[i,2];

    return newMesh;

In [5]:
# Load the sphere
sphere=open_ply("sphere.ply");
print("vertices: %i"%sphere["np"]);
print("triangles: %i"%sphere["nt"]);

# Test: scale a single sphere
# ellip=scale_mesh(sphere,1,2,3);
# save_ply(ellip,"ellip.ply");

# Produce ellipsoids with a=1, b=0.8:0.1:1.2 and c=b:0.1:1.2
a=1;
for b in frange(0.8,1.2,0.1):
    for c in frange(b,1.2,0.1):
        ellip=scale_mesh(sphere,a,b,c);

        # Save in STL
        fileName="ellipsoids/ellip.a%g_b%g_c%g.stl"%(a,b,c);
        save_stl(ellip,fileName);
        
        #Save in PLY
        #fileName="ellipsoids/ellip.a%g_b%g_c%g.ply"%(a,b,c);
        #save_ply(ellip,fileName);


vertices: 9722
triangles: 19440
